**ÍNDICES AGROCLIMÁTICOS**

In [57]:
import datetime as dt
import pandas as pd
import string as str
import numpy as np

In [58]:
df_data_all_stations = pd.read_csv('.csv', header = 0,delimiter=';', encoding='latin-1')

In [59]:
df_data_all_stations.info()
df_data_all_stations.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21434 entries, 0 to 21433
Data columns (total 27 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Date & Time                       21434 non-null  object 
 1   ID                                21434 non-null  int64  
 2   Alta velocidad del viento - km/h  21433 non-null  float64
 3   BarÃ³metro - mb                   18449 non-null  float64
 4   Bulbo HÃºmedo - Â°C               18429 non-null  float64
 5   DÃ­as-grado de calentamiento      18429 non-null  float64
 6   DÃ­as-grado de enfriamiento       18429 non-null  float64
 7   ET - mm                           21433 non-null  float64
 8   EnergÃ­a solar - Ly               18429 non-null  float64
 9   Hum - %                           19690 non-null  float64
 10  Humedad del suelo  cm - cb        18408 non-null  float64
 11  Humedad del suelo  cm - cb.1      13789 non-null  float64
 12  Hume

,Date & Time,ID,Alta velocidad del viento - km/h,BarÃ³metro - mb,Bulbo HÃºmedo - Â°C,DÃ­as-grado de calentamiento,DÃ­as-grado de enfriamiento,ET - mm,EnergÃ­a solar - Ly,Hum - %,...,THSW Ã­ndice - Â°C,THW Index - Â°C,Tasa de lluvia - mm/h,Temp - Â°C,Temp. Baja - Â°C,Tempalta,Velocidad del viento - km/h,Viento Corriente - km,Viento FrÃ­o - Â°C,Ãndice de calor - Â°C
0,1/01/2024 00:00,169484,4.8,1025.0,14.9,0.037,0.0,0.00,0.0,77.0,...,16.7,17.5,0.0,17.4,17.4,17.7,1.6,1.61,17.4,17.5
1,1/01/2024 00:00,169523,3.2,972.9,16.2,0.012,0.0,0.00,0.0,84.0,...,17.2,18.4,0.0,18.1,17.8,18.3,0.0,0.00,18.1,18.4
2,1/01/2024 00:00,169524,3.2,1023.6,15.0,0.021,0.0,0.00,0.0,75.0,...,16.7,17.8,0.0,17.8,17.7,17.9,0.0,0.00,17.8,17.8
3,1/01/2024 00:00,169529,8.0,946.3,14.4,0.030,0.0,0.03,0.0,73.0,...,16.7,17.5,0.0,17.6,17.4,17.8,3.2,3.22,17.6,17.5
4,1/01/2024 01:00,169484,4.8,1024.6,15.1,0.044,0.0,0.00,0.0,80.0,...,16.1,17.4,0.0,17.3,17.0,17.6,1.6,1.61,17.3,17.4


In [60]:
df_lluvia = df_data_all_stations

In [61]:
# Convertir 'Date & Time' a formato datetime manejando diferentes formatos de fecha
df_lluvia['Date & Time'] = pd.to_datetime(df_lluvia['Date & Time'], errors='coerce', dayfirst=True)

# Filtrar el DataFrame para conservar solo las columnas necesarias
df_lluvia2 = df_lluvia[['Date & Time', 'ID', 'Lluvia - mm']].copy()

# Asegurarse de que 'Lluvia - mm' es de tipo float para poder sumar
df_lluvia2['Lluvia - mm'] = pd.to_numeric(df_lluvia2['Lluvia - mm'], errors='coerce')

# Crear una columna con la fecha
df_lluvia2['Dia'] = df_lluvia2['Date & Time'].dt.to_period('D')

# Agrupar por 'ID' y 'Dia' y sumar la lluvia diaria
df_lluvia_diaria = df_lluvia2.groupby(['ID', 'Dia'])['Lluvia - mm'].sum().reset_index()

# Convertir 'Dia' a formato datetime para la columna 'Date & Time'
df_lluvia_diaria['Date & Time'] = df_lluvia_diaria['Dia'].dt.to_timestamp()

In [62]:
df_lluvia_diaria.info()
df_lluvia_diaria.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   ID           900 non-null    int64         
 1   Dia          900 non-null    period[D]     
 2   Lluvia - mm  900 non-null    float64       
 3   Date & Time  900 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), period[D](1)
memory usage: 28.3 KB


,ID,Dia,Lluvia - mm,Date & Time
0,169484,2023-11-14,2.6,2023-11-14
1,169484,2023-11-15,0.0,2023-11-15
2,169484,2023-11-16,0.8,2023-11-16
3,169484,2023-11-17,0.0,2023-11-17
4,169484,2023-11-18,0.0,2023-11-18


**No. de días secos Ciclo de cultivo Por Estación**
Total number of dry days during the corn production season (Mayo - Septiembre). 
A dry day occurs when the amount of daily precipitation drops below 1 mm.

In [63]:
dia_seco = df_lluvia_diaria

Día seco = 1 (sí)
Día lluvioso = 0(no)

In [64]:
# Crear la columna 'dia_seco' con la condición especificada
dia_seco['dia_seco'] = dia_seco['Lluvia - mm'].apply(lambda x: 1 if x <= 1 else 0)

In [65]:
dia_seco.to_csv('.csv', index=False, sep=';')

In [66]:
df_tabla_con_dia_seco = dia_seco

In [67]:
df_dia_humedo = dia_seco

In [68]:
df_dia_humedo.head()

,ID,Dia,Lluvia - mm,Date & Time,dia_seco
0,169484,2023-11-14,2.6,2023-11-14,0
1,169484,2023-11-15,0.0,2023-11-15,1
2,169484,2023-11-16,0.8,2023-11-16,1
3,169484,2023-11-17,0.0,2023-11-17,1
4,169484,2023-11-18,0.0,2023-11-18,1


In [69]:
dia_seco.info()
dia_seco.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   ID           900 non-null    int64         
 1   Dia          900 non-null    period[D]     
 2   Lluvia - mm  900 non-null    float64       
 3   Date & Time  900 non-null    datetime64[ns]
 4   dia_seco     900 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(2), period[D](1)
memory usage: 35.3 KB


,ID,Dia,Lluvia - mm,Date & Time,dia_seco
0,169484,2023-11-14,2.6,2023-11-14,0
1,169484,2023-11-15,0.0,2023-11-15,1
2,169484,2023-11-16,0.8,2023-11-16,1
3,169484,2023-11-17,0.0,2023-11-17,1
4,169484,2023-11-18,0.0,2023-11-18,1


In [70]:
# Filtrar el DataFrame para incluir solo los meses de mayo a septiembre
dia_seco_filtrado = dia_seco[dia_seco['Date & Time'].dt.month.isin([5, 6, 7, 8, 9])]

# Crear una columna de año
dia_seco_filtrado['Year'] = dia_seco_filtrado['Date & Time'].dt.year

# Agrupar por año e ID y contar los días secos
conteo_dias_secos = dia_seco_filtrado[dia_seco_filtrado['dia_seco'] == 1].groupby(['Year', 'ID']).size().reset_index(name='Count_dias_secos')


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dia_seco_filtrado['Year'] = dia_seco_filtrado['Date & Time'].dt.year


In [71]:
conteo_dias_secos.info()
conteo_dias_secos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   Year              5 non-null      int32
 1   ID                5 non-null      int64
 2   Count_dias_secos  5 non-null      int64
dtypes: int32(1), int64(2)
memory usage: 232.0 bytes


,Year,ID,Count_dias_secos
0,2024,169484,24
1,2024,169523,20
2,2024,169524,22
3,2024,169529,22
4,2024,178223,26


In [72]:
conteo_dias_secos.to_csv('.csv', index=False, sep=';')

**No. de días secos Por mes Por Estación**
Total number of dry days 
A dry day occurs when the amount of daily precipitation drops below 1 mm.

In [73]:
# Crear columnas de año y mes
dia_seco['Year'] = dia_seco['Date & Time'].dt.year
dia_seco['Mes'] = dia_seco['Date & Time'].dt.month

# Agrupar por año, mes e ID y contar los días secos
conteo_dias_secos_por_mes = dia_seco[dia_seco['dia_seco'] == 1].groupby(['Year', 'Mes', 'ID']).size().reset_index(name='Count_dias_secos_por_mes')

In [74]:
conteo_dias_secos_por_mes.info()
conteo_dias_secos_por_mes.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype
---  ------                    --------------  -----
 0   Year                      33 non-null     int32
 1   Mes                       33 non-null     int32
 2   ID                        33 non-null     int64
 3   Count_dias_secos_por_mes  33 non-null     int64
dtypes: int32(2), int64(2)
memory usage: 924.0 bytes


,Year,Mes,ID,Count_dias_secos_por_mes
0,2023,11,169484,16
1,2023,11,169523,17
2,2023,11,169524,17
3,2023,11,169529,16
4,2023,12,169484,30


In [75]:
# Filtrar las filas donde Count_dias_secos_por_mes es mayor a 10
alertas = conteo_dias_secos_por_mes[conteo_dias_secos_por_mes['Count_dias_secos_por_mes'] > 10]

# Generar las alertas
for index, row in alertas.iterrows():
    print(f"Alerta: El ID {row['ID']} ya tiene {row['Count_dias_secos_por_mes']} días secos en el mes {row['Mes']} del año {row['Year']}. ¡Ya son muchos días secos!")

Alerta: El ID 169484 ya tiene 16 días secos en el mes 11 del año 2023. ¡Ya son muchos días secos!
Alerta: El ID 169523 ya tiene 17 días secos en el mes 11 del año 2023. ¡Ya son muchos días secos!
Alerta: El ID 169524 ya tiene 17 días secos en el mes 11 del año 2023. ¡Ya son muchos días secos!
Alerta: El ID 169529 ya tiene 16 días secos en el mes 11 del año 2023. ¡Ya son muchos días secos!
Alerta: El ID 169484 ya tiene 30 días secos en el mes 12 del año 2023. ¡Ya son muchos días secos!
Alerta: El ID 169523 ya tiene 30 días secos en el mes 12 del año 2023. ¡Ya son muchos días secos!
Alerta: El ID 169524 ya tiene 28 días secos en el mes 12 del año 2023. ¡Ya son muchos días secos!
Alerta: El ID 169529 ya tiene 28 días secos en el mes 12 del año 2023. ¡Ya son muchos días secos!
Alerta: El ID 169484 ya tiene 30 días secos en el mes 1 del año 2024. ¡Ya son muchos días secos!
Alerta: El ID 169523 ya tiene 31 días secos en el mes 1 del año 2024. ¡Ya son muchos días secos!
Alerta: El ID 169524 y

In [76]:
conteo_dias_secos_por_mes.to_csv('.csv', index=False, sep=';')

**No. de periodos secos Por Ciclo de Cultivo Por Estación**
Total number of dry spells during the maize production season (September to April). 
A dry spell is defined as a period of five or more consecutive dry days immediately followed by a rainy day.

In [77]:
periodo_seco = dia_seco

In [78]:
periodo_seco.info()
periodo_seco.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   ID           900 non-null    int64         
 1   Dia          900 non-null    period[D]     
 2   Lluvia - mm  900 non-null    float64       
 3   Date & Time  900 non-null    datetime64[ns]
 4   dia_seco     900 non-null    int64         
 5   Year         900 non-null    int32         
 6   Mes          900 non-null    int32         
dtypes: datetime64[ns](1), float64(1), int32(2), int64(2), period[D](1)
memory usage: 42.3 KB


,ID,Dia,Lluvia - mm,Date & Time,dia_seco,Year,Mes
0,169484,2023-11-14,2.6,2023-11-14,0,2023,11
1,169484,2023-11-15,0.0,2023-11-15,1,2023,11
2,169484,2023-11-16,0.8,2023-11-16,1,2023,11
3,169484,2023-11-17,0.0,2023-11-17,1,2023,11
4,169484,2023-11-18,0.0,2023-11-18,1,2023,11


In [79]:
# Obtener el año más reciente en el DataFrame
año_presente = periodo_seco['Year'].max()
año_anterior = año_presente - 1

# Filtrar por los meses de septiembre del año anterior a abril del año presente
filtro_mes = ((periodo_seco['Year'] == año_anterior) & (periodo_seco['Mes'] >= 9)) | ((periodo_seco['Year'] == año_presente) & (periodo_seco['Mes'] <= 4))
df_filtrado_mes = periodo_seco[filtro_mes]

In [80]:
df_filtrado_mes.to_csv('.csv', index=False, sep=';')

In [81]:
df_filtrado_mes.info()
df_filtrado_mes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 772 entries, 0 to 871
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   ID           772 non-null    int64         
 1   Dia          772 non-null    period[D]     
 2   Lluvia - mm  772 non-null    float64       
 3   Date & Time  772 non-null    datetime64[ns]
 4   dia_seco     772 non-null    int64         
 5   Year         772 non-null    int32         
 6   Mes          772 non-null    int32         
dtypes: datetime64[ns](1), float64(1), int32(2), int64(2), period[D](1)
memory usage: 42.2 KB


,ID,Dia,Lluvia - mm,Date & Time,dia_seco,Year,Mes
0,169484,2023-11-14,2.6,2023-11-14,0,2023,11
1,169484,2023-11-15,0.0,2023-11-15,1,2023,11
2,169484,2023-11-16,0.8,2023-11-16,1,2023,11
3,169484,2023-11-17,0.0,2023-11-17,1,2023,11
4,169484,2023-11-18,0.0,2023-11-18,1,2023,11


In [82]:
# Ordenar el DataFrame por 'ID' y 'Date & Time'
df_filtrado_mes = df_filtrado_mes.sort_values(by=['ID', 'Date & Time'])

# Inicializar un DataFrame para guardar los resultados
resultado_periodo_seco = pd.DataFrame(columns=['ID', 'No. de periodos secos'])

# Inicializar variables
prev_id = None
dias_sin_lluvia = 0
num_periodos_secos = 0

# Lista temporal para almacenar resultados
resultados_temp = []

# Iterar sobre las filas del DataFrame
for index, row in df_filtrado_mes.iterrows():
    current_id = row['ID']
    
    if current_id != prev_id and prev_id is not None:
        # Guardar el conteo de periodos secos para el ID anterior
        resultados_temp.append({'ID': prev_id, 'No. de periodos secos': num_periodos_secos})
        # Reiniciar el contador de periodos secos
        num_periodos_secos = 0
        dias_sin_lluvia = 0
    
    if row['dia_seco'] == 1:
        dias_sin_lluvia += 1
    else:
        if dias_sin_lluvia >= 5:
            num_periodos_secos += 1
        dias_sin_lluvia = 0
    
    prev_id = current_id

# Agregar el conteo para el último ID
if prev_id is not None:
    resultados_temp.append({'ID': prev_id, 'No. de periodos secos': num_periodos_secos})

# Crear el DataFrame final con los resultados
resultado_periodo_seco = pd.DataFrame(resultados_temp)

# Convertir el ID a int64
resultado_periodo_seco['ID'] = resultado_periodo_seco['ID'].astype('int64')

# Mostrar el DataFrame resultado
print(resultado_periodo_seco)


       ID  No. de periodos secos
0  169484                      5
1  169523                      2
2  169524                      4
3  169529                      6
4  178223                      1


In [83]:
resultado_periodo_seco.info()
resultado_periodo_seco.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   ID                     5 non-null      int64
 1   No. de periodos secos  5 non-null      int64
dtypes: int64(2)
memory usage: 212.0 bytes


,ID,No. de periodos secos
0,169484,5
1,169523,2
2,169524,4
3,169529,6
4,178223,1


In [84]:
resultado_periodo_seco.to_csv('.csv', index=False, sep=';')

Comprobación de rangos de sin lluvia

In [85]:
import numpy as np

# Ordenar el DataFrame por 'ID' y 'Date & Time'
df_filtrado_mes = df_filtrado_mes.sort_values(by=['ID', 'Date & Time'])

# Inicializar listas para guardar los resultados
ids = []
inicio_rango = []
fin_rango = []
duracion = []

# Inicializar variables
dias_sin_lluvia = 0
inicio_sin_lluvia = None
prev_id = None

# Iterar sobre las filas del DataFrame
for index, row in df_filtrado_mes.iterrows():
    current_id = row['ID']
    dia_seco = row['dia_seco']
    
    if current_id != prev_id:
        # Guardar el rango de días sin lluvia para el ID anterior
        if prev_id is not None:
            ids.append(prev_id)
            inicio_rango.append(inicio_sin_lluvia)
            fin_rango.append(prev_fecha)
            duracion.append(dias_sin_lluvia)
        
        # Reiniciar variables al cambiar de ID
        dias_sin_lluvia = 0
        inicio_sin_lluvia = None
    
    if dia_seco == 1:
        if inicio_sin_lluvia is None:
            inicio_sin_lluvia = row['Date & Time']
        dias_sin_lluvia += 1
        prev_fecha = row['Date & Time']
    else:
        if inicio_sin_lluvia is not None:
            # Guardar el rango de días sin lluvia cuando se encuentra un día con lluvia
            ids.append(prev_id)
            inicio_rango.append(inicio_sin_lluvia)
            fin_rango.append(prev_fecha)
            duracion.append(dias_sin_lluvia)
            # Reiniciar variables para el nuevo período de días sin lluvia
            dias_sin_lluvia = 0
            inicio_sin_lluvia = None
    
    prev_id = current_id

# Agregar el rango de días sin lluvia para el último ID
if prev_id is not None and inicio_sin_lluvia is not None:
    ids.append(prev_id)
    inicio_rango.append(inicio_sin_lluvia)
    fin_rango.append(prev_fecha)
    duracion.append(dias_sin_lluvia)

# Convertir las listas a un DataFrame
resultado_dias_sin_lluvia = pd.DataFrame({
    'ID': ids,
    'Inicio del rango': inicio_rango,
    'Fin del rango': fin_rango,
    'Duración': duracion
})

# Convertir el ID a int64
resultado_dias_sin_lluvia['ID'] = resultado_dias_sin_lluvia['ID'].astype('int64')

# Mostrar el DataFrame resultado
print(resultado_dias_sin_lluvia)



        ID Inicio del rango Fin del rango  Duración
0   169484       2023-11-15    2023-12-04        20
1   169484       2023-12-06    2024-01-28        54
2   169484       2024-01-30    2024-03-17        48
3   169484       2024-03-19    2024-04-07        20
4   169484       2024-04-10    2024-04-28        19
5   169484              NaT    2024-04-28         0
6   169523       2023-11-14    2023-12-29        46
7   169523       2023-12-31    2024-04-28       120
8   169523              NaT    2024-04-28         0
9   169524       2023-11-14    2023-12-03        20
10  169524       2023-12-06    2023-12-10         5
11  169524       2023-12-12    2024-01-27        47
12  169524       2024-01-29    2024-04-29        92
13  169524              NaT    2024-04-29         0
14  169529       2023-11-14    2023-11-14         1
15  169529       2023-11-16    2023-12-03        18
16  169529       2023-12-06    2023-12-10         5
17  169529       2023-12-12    2024-01-27        47
18  169529  

**No de períodos secos Por Mes por Estación**

In [86]:
periodo_seco_mes = df_tabla_con_dia_seco

In [87]:
periodo_seco_mes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   ID           900 non-null    int64         
 1   Dia          900 non-null    period[D]     
 2   Lluvia - mm  900 non-null    float64       
 3   Date & Time  900 non-null    datetime64[ns]
 4   dia_seco     900 non-null    int64         
 5   Year         900 non-null    int32         
 6   Mes          900 non-null    int32         
dtypes: datetime64[ns](1), float64(1), int32(2), int64(2), period[D](1)
memory usage: 42.3 KB


In [88]:
# Ordenar el DataFrame por 'ID', 'Year', 'Mes' y 'Date & Time'
df_orden_mes = periodo_seco_mes.sort_values(by=['ID', 'Year', 'Mes', 'Date & Time'])

# Inicializar un DataFrame para guardar los resultados
resultado_periodo_seco_mes = pd.DataFrame(columns=['ID', 'Year', 'Mes', 'No. de periodos secos'])

# Inicializar variables
prev_id = None
prev_year = None
prev_mes = None
dias_sin_lluvia = 0
num_periodos_secos = 0

# Lista temporal para almacenar resultados
resultados_temp = []

# Iterar sobre las filas del DataFrame
for index, row in df_orden_mes.iterrows():
    current_id = row['ID']
    current_year = row['Year']
    current_mes = row['Mes']
    
    if (current_id != prev_id or current_year != prev_year or current_mes != prev_mes) and prev_id is not None:
        # Guardar el conteo de periodos secos para el ID, año y mes anterior
        resultados_temp.append({
            'ID': prev_id, 
            'Year': prev_year, 
            'Mes': prev_mes, 
            'No. de periodos secos': num_periodos_secos
        })
        # Reiniciar el contador de periodos secos
        num_periodos_secos = 0
        dias_sin_lluvia = 0
    
    if row['dia_seco'] == 1:
        dias_sin_lluvia += 1
    else:
        if dias_sin_lluvia >= 5:
            num_periodos_secos += 1
        dias_sin_lluvia = 0
    
    prev_id = current_id
    prev_year = current_year
    prev_mes = current_mes

# Agregar el conteo para el último ID, año y mes
if prev_id is not None:
    resultados_temp.append({
        'ID': prev_id, 
        'Year': prev_year, 
        'Mes': prev_mes, 
        'No. de periodos secos': num_periodos_secos
    })

# Crear el DataFrame final con los resultados
resultado_periodo_seco_mes = pd.DataFrame(resultados_temp)

# Convertir el ID a int64
resultado_periodo_seco_mes['ID'] = resultado_periodo_seco_mes['ID'].astype('int64')

# Mostrar el DataFrame resultado
print(resultado_periodo_seco_mes)


        ID  Year  Mes  No. de periodos secos
0   169484  2023   11                      0
1   169484  2023   12                      0
2   169484  2024    1                      1
3   169484  2024    2                      0
4   169484  2024    3                      1
5   169484  2024    4                      2
6   169484  2024    5                      1
7   169523  2023   11                      0
8   169523  2023   12                      1
9   169523  2024    1                      0
10  169523  2024    2                      0
11  169523  2024    3                      0
12  169523  2024    4                      1
13  169523  2024    5                      1
14  169524  2023   11                      0
15  169524  2023   12                      1
16  169524  2024    1                      1
17  169524  2024    2                      0
18  169524  2024    3                      0
19  169524  2024    4                      1
20  169524  2024    5                      1
21  169529

In [89]:
# Verificar que el DataFrame contiene las columnas necesarias
required_columns = ['ID', 'Year', 'Mes', 'Date & Time', 'dia_seco']
for col in required_columns:
    if col not in periodo_seco_mes.columns:
        raise ValueError(f"Falta la columna necesaria: {col}")

# Ordenar el DataFrame por 'ID', 'Year', 'Mes' y 'Date & Time'
df_orden_mes = periodo_seco_mes.sort_values(by=['ID', 'Year', 'Mes', 'Date & Time'])

# Inicializar un DataFrame para guardar los resultados
resultado_periodo_seco_mes = pd.DataFrame(columns=['ID', 'Year', 'Mes', 'No. de periodos secos'])

# Inicializar variables
prev_id = None
prev_year = None
prev_mes = None
dias_sin_lluvia = 0
num_periodos_secos = 0

# Lista temporal para almacenar resultados
resultados_temp = []

# Iterar sobre las filas del DataFrame
for index, row in df_orden_mes.iterrows():
    current_id = row['ID']
    current_year = row['Year']
    current_mes = row['Mes']
    
    if (current_id != prev_id or current_year != prev_year or current_mes != prev_mes) and prev_id is not None:
        # Guardar el conteo de periodos secos para el ID, año y mes anterior
        resultados_temp.append({
            'ID': prev_id, 
            'Year': prev_year, 
            'Mes': prev_mes, 
            'No. de periodos secos': num_periodos_secos
        })
        # Reiniciar el contador de periodos secos y días sin lluvia
        num_periodos_secos = 0
        dias_sin_lluvia = 0
    
    if row['dia_seco'] == 1:
        dias_sin_lluvia += 1
    else:
        if dias_sin_lluvia >= 5:
            num_periodos_secos += 1
        dias_sin_lluvia = 0
    
    prev_id = current_id
    prev_year = current_year
    prev_mes = current_mes

# Agregar el conteo para el último ID, año y mes
if prev_id is not None:
    if dias_sin_lluvia >= 5:
        num_periodos_secos += 1
    resultados_temp.append({
        'ID': prev_id, 
        'Year': prev_year, 
        'Mes': prev_mes, 
        'No. de periodos secos': num_periodos_secos
    })

# Crear el DataFrame final con los resultados
resultado_periodo_seco_mes = pd.DataFrame(resultados_temp)

# Convertir el ID a int64
resultado_periodo_seco_mes['ID'] = resultado_periodo_seco_mes['ID'].astype('int64')

# Mostrar el DataFrame resultado
print(resultado_periodo_seco_mes)


        ID  Year  Mes  No. de periodos secos
0   169484  2023   11                      0
1   169484  2023   12                      0
2   169484  2024    1                      1
3   169484  2024    2                      0
4   169484  2024    3                      1
5   169484  2024    4                      2
6   169484  2024    5                      1
7   169523  2023   11                      0
8   169523  2023   12                      1
9   169523  2024    1                      0
10  169523  2024    2                      0
11  169523  2024    3                      0
12  169523  2024    4                      1
13  169523  2024    5                      1
14  169524  2023   11                      0
15  169524  2023   12                      1
16  169524  2024    1                      1
17  169524  2024    2                      0
18  169524  2024    3                      0
19  169524  2024    4                      1
20  169524  2024    5                      1
21  169529

Comprobación del rango del periodo seco por mes 

In [90]:
# Verificar que el DataFrame contiene las columnas necesarias
required_columns = ['ID', 'Year', 'Mes', 'Date & Time', 'dia_seco']
for col in required_columns:
    if col not in periodo_seco_mes.columns:
        raise ValueError(f"Falta la columna necesaria: {col}")

# Ordenar el DataFrame por 'ID', 'Year', 'Mes' y 'Date & Time'
df_orden_mes = periodo_seco_mes.sort_values(by=['ID', 'Year', 'Mes', 'Date & Time'])

# Inicializar un DataFrame para guardar los resultados
resultado_periodo_seco_mes = pd.DataFrame(columns=['ID', 'Year', 'Mes', 'Inicio periodo seco', 'Fin periodo seco', 'Duración periodo seco', 'Dias sin lluvia'])

# Inicializar variables
prev_id = None
prev_year = None
prev_mes = None
dias_sin_lluvia = 0
inicio_periodo = None

# Lista temporal para almacenar resultados
resultados_temp = []

# Iterar sobre las filas del DataFrame
for index, row in df_orden_mes.iterrows():
    current_id = row['ID']
    current_year = row['Year']
    current_mes = row['Mes']
    current_date = row['Date & Time']
    
    if (current_id != prev_id or current_year != prev_year or current_mes != prev_mes) and prev_id is not None:
        # Guardar el conteo de periodos secos para el ID, año y mes anterior
        if dias_sin_lluvia >= 5:
            resultados_temp.append({
                'ID': prev_id, 
                'Year': prev_year, 
                'Mes': prev_mes, 
                'Inicio periodo seco': inicio_periodo,
                'Fin periodo seco': prev_date,
                'Duración periodo seco': dias_sin_lluvia,
                'Dias sin lluvia': dias_sin_lluvia
            })
        # Reiniciar el contador de periodos secos y días sin lluvia
        dias_sin_lluvia = 0
        inicio_periodo = None
    
    if row['dia_seco'] == 1:
        if dias_sin_lluvia == 0:
            inicio_periodo = current_date
        dias_sin_lluvia += 1
    else:
        if dias_sin_lluvia >= 5:
            resultados_temp.append({
                'ID': current_id, 
                'Year': current_year, 
                'Mes': current_mes, 
                'Inicio periodo seco': inicio_periodo,
                'Fin periodo seco': current_date - pd.Timedelta(days=1),
                'Duración periodo seco': dias_sin_lluvia,
                'Dias sin lluvia': dias_sin_lluvia
            })
        dias_sin_lluvia = 0
        inicio_periodo = None
    
    prev_id = current_id
    prev_year = current_year
    prev_mes = current_mes
    prev_date = current_date

# Agregar el conteo para el último ID, año y mes
if prev_id is not None and dias_sin_lluvia >= 5:
    resultados_temp.append({
        'ID': prev_id, 
        'Year': prev_year, 
        'Mes': prev_mes, 
        'Inicio periodo seco': inicio_periodo,
        'Fin periodo seco': prev_date,
        'Duración periodo seco': dias_sin_lluvia,
        'Dias sin lluvia': dias_sin_lluvia
    })

# Crear el DataFrame final con los resultados
resultado_periodo_seco_mes = pd.DataFrame(resultados_temp)

# Convertir el ID a int64
resultado_periodo_seco_mes['ID'] = resultado_periodo_seco_mes['ID'].astype('int64')


In [91]:
resultado_periodo_seco_mes.info()
resultado_periodo_seco_mes.head(50)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     43 non-null     int64         
 1   Year                   43 non-null     int64         
 2   Mes                    43 non-null     int64         
 3   Inicio periodo seco    43 non-null     datetime64[ns]
 4   Fin periodo seco       43 non-null     datetime64[ns]
 5   Duración periodo seco  43 non-null     int64         
 6   Dias sin lluvia        43 non-null     int64         
dtypes: datetime64[ns](2), int64(5)
memory usage: 2.5 KB


,ID,Year,Mes,Inicio periodo seco,Fin periodo seco,Duración periodo seco,Dias sin lluvia
0,169484,2023,11,2023-11-15,2023-11-30,16,16
1,169484,2023,12,2023-12-06,2023-12-31,26,26
2,169484,2024,1,2024-01-01,2024-01-28,28,28
3,169484,2024,2,2024-02-01,2024-02-29,29,29
4,169484,2024,3,2024-03-01,2024-03-17,17,17
5,169484,2024,3,2024-03-19,2024-03-31,13,13
6,169484,2024,4,2024-04-01,2024-04-07,7,7
7,169484,2024,4,2024-04-10,2024-04-28,19,19
8,169484,2024,5,2024-05-01,2024-05-15,15,15
9,169484,2024,5,2024-05-17,2024-05-25,9,9


**Script en uso para determinar no. de periodos secos cada mes**

In [92]:
# Verificar que el DataFrame contiene las columnas necesarias
required_columns = ['ID', 'Year', 'Mes', 'Date & Time', 'dia_seco']
for col in required_columns:
    if col not in periodo_seco_mes.columns:
        raise ValueError(f"Falta la columna necesaria: {col}")

# Ordenar el DataFrame por 'ID', 'Year', 'Mes' y 'Date & Time'
df_orden_mes = periodo_seco_mes.sort_values(by=['ID', 'Year', 'Mes', 'Date & Time'])

# Inicializar un DataFrame para guardar los resultados
resultado_periodo_seco_mes = pd.DataFrame(columns=['ID', 'Year', 'Mes', 'Inicio periodo seco', 'Fin periodo seco', 'Duración periodo seco', 'Dias sin lluvia', 'periodo_seco_mes'])

# Inicializar variables
prev_id = None
prev_year = None
prev_mes = None
dias_sin_lluvia = 0
inicio_periodo = None

# Lista temporal para almacenar resultados
resultados_temp = []

# Iterar sobre las filas del DataFrame
for index, row in df_orden_mes.iterrows():
    current_id = row['ID']
    current_year = row['Year']
    current_mes = row['Mes']
    current_date = row['Date & Time']
    
    if (current_id != prev_id or current_year != prev_year or current_mes != prev_mes) and prev_id is not None:
        # Guardar el conteo de periodos secos para el ID, año y mes anterior
        if dias_sin_lluvia >= 5:
            resultados_temp.append({
                'ID': prev_id, 
                'Year': prev_year, 
                'Mes': prev_mes, 
                'Inicio periodo seco': inicio_periodo,
                'Fin periodo seco': prev_date,
                'Duración periodo seco': dias_sin_lluvia,
                'Dias sin lluvia': dias_sin_lluvia,
                'periodo_seco_mes': 1
            })
        # Reiniciar el contador de periodos secos y días sin lluvia
        dias_sin_lluvia = 0
        inicio_periodo = None
    
    if row['dia_seco'] == 1:
        if dias_sin_lluvia == 0:
            inicio_periodo = current_date
        dias_sin_lluvia += 1
    else:
        if dias_sin_lluvia >= 5:
            resultados_temp.append({
                'ID': current_id, 
                'Year': current_year, 
                'Mes': current_mes, 
                'Inicio periodo seco': inicio_periodo,
                'Fin periodo seco': current_date - pd.Timedelta(days=1),
                'Duración periodo seco': dias_sin_lluvia,
                'Dias sin lluvia': dias_sin_lluvia,
                'periodo_seco_mes': 1
            })
        dias_sin_lluvia = 0
        inicio_periodo = None
    
    prev_id = current_id
    prev_year = current_year
    prev_mes = current_mes
    prev_date = current_date

# Agregar el conteo para el último ID, año y mes
if prev_id is not None and dias_sin_lluvia >= 5:
    resultados_temp.append({
        'ID': prev_id, 
        'Year': prev_year, 
        'Mes': prev_mes, 
        'Inicio periodo seco': inicio_periodo,
        'Fin periodo seco': prev_date,
        'Duración periodo seco': dias_sin_lluvia,
        'Dias sin lluvia': dias_sin_lluvia,
        'periodo_seco_mes': 1
    })

# Crear el DataFrame final con los resultados
resultado_periodo_seco_mes = pd.DataFrame(resultados_temp)

# Añadir la columna 'periodo_seco_mes' para todos los registros donde 'Dias sin lluvia' es menor a 5
for index, row in df_orden_mes.iterrows():
    if row['dia_seco'] == 1:
        dias_sin_lluvia += 1
    else:
        if dias_sin_lluvia < 5:
            resultados_temp.append({
                'ID': current_id, 
                'Year': current_year, 
                'Mes': current_mes, 
                'Inicio periodo seco': inicio_periodo,
                'Fin periodo seco': current_date - pd.Timedelta(days=1),
                'Duración periodo seco': dias_sin_lluvia,
                'Dias sin lluvia': dias_sin_lluvia,
                'periodo_seco_mes': 0
            })

# Convertir el ID a int64
resultado_periodo_seco_mes['ID'] = resultado_periodo_seco_mes['ID'].astype('int64')


In [93]:
resultado_periodo_seco_mes.head(15)

,ID,Year,Mes,Inicio periodo seco,Fin periodo seco,Duración periodo seco,Dias sin lluvia,periodo_seco_mes
0,169484,2023,11,2023-11-15,2023-11-30,16,16,1
1,169484,2023,12,2023-12-06,2023-12-31,26,26,1
2,169484,2024,1,2024-01-01,2024-01-28,28,28,1
3,169484,2024,2,2024-02-01,2024-02-29,29,29,1
4,169484,2024,3,2024-03-01,2024-03-17,17,17,1
5,169484,2024,3,2024-03-19,2024-03-31,13,13,1
6,169484,2024,4,2024-04-01,2024-04-07,7,7,1
7,169484,2024,4,2024-04-10,2024-04-28,19,19,1
8,169484,2024,5,2024-05-01,2024-05-15,15,15,1
9,169484,2024,5,2024-05-17,2024-05-25,9,9,1


In [94]:
# Agrupar por ID, Year y Mes y sumar la columna periodo_seco_mes
cantidad_periodo_seco_mes = resultado_periodo_seco_mes.groupby(['ID', 'Year', 'Mes']).agg({'periodo_seco_mes': 'sum'}).reset_index()

# Renombrar la columna periodo_seco_mes a Cantidad de periodos secos
cantidad_periodo_seco_mes = cantidad_periodo_seco_mes.rename(columns={'periodo_seco_mes': 'Cantidad de periodos secos'})

# Convertir el ID a int64
cantidad_periodo_seco_mes['ID'] = cantidad_periodo_seco_mes['ID'].astype('int64')

# Mostrar el DataFrame resultado
print(cantidad_periodo_seco_mes)


        ID  Year  Mes  Cantidad de periodos secos
0   169484  2023   11                           1
1   169484  2023   12                           1
2   169484  2024    1                           1
3   169484  2024    2                           1
4   169484  2024    3                           2
5   169484  2024    4                           2
6   169484  2024    5                           2
7   169523  2023   11                           1
8   169523  2023   12                           1
9   169523  2024    1                           1
10  169523  2024    2                           1
11  169523  2024    3                           1
12  169523  2024    4                           1
13  169523  2024    5                           2
14  169524  2023   11                           1
15  169524  2023   12                           2
16  169524  2024    1                           1
17  169524  2024    2                           1
18  169524  2024    3                           1


In [95]:
cantidad_periodo_seco_mes.info()
cantidad_periodo_seco_mes.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 4 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   ID                          33 non-null     int64
 1   Year                        33 non-null     int64
 2   Mes                         33 non-null     int64
 3   Cantidad de periodos secos  33 non-null     int64
dtypes: int64(4)
memory usage: 1.2 KB


,ID,Year,Mes,Cantidad de periodos secos
0,169484,2023,11,1
1,169484,2023,12,1
2,169484,2024,1,1
3,169484,2024,2,1
4,169484,2024,3,2


In [96]:
cantidad_periodo_seco_mes.to_csv('.csv', index=False, sep=';')

**Duración promedio de los períodos secos**
Average duration of dry periods during the maize production campaign (September to April). 
This index describes the duration in days of dry spells.

Este índice proporciona una perspectiva cuantitativa sobre la cantidad de días en los que no hay precipitaciones. Un valor alto indica que hay períodos prolongados sin lluvia, lo que puede ser problemático para el crecimiento de los cultivos y requerir medidas de riego adicionales.


Por Ciclo de cultivo para cada estación: 
Ej:
ID 169484 tiene 5 periodos secos

|    Periodo    | Duración(días)|
| ------------- | ------------- |
| 1             |20             |
| 2             |54             |
| 3             |48             |
| 4             |20             |
| 5             |19             |
  

Duración promedio de los períodos secos: 32.2 días


In [97]:
###hay que eliminar antes el 0 de duración
# Calcular el promedio de duración por estación
promedio_por_estacion = resultado_dias_sin_lluvia.groupby('ID')['Duración'].mean()

# Mostrar el resultado
print("Promedio de duración por estación:")
print(promedio_por_estacion)


Promedio de duración por estación:
ID
169484    26.833333
169523    55.333333
169524    32.800000
169529    17.777778
178223    94.000000
Name: Duración, dtype: float64


In [98]:
# Convertir el diccionario en DataFrame
df_promedio = pd.DataFrame(list(promedio_por_estacion.items()), columns=['ID', 'Promedio'])

# Establecer 'ID' como índice
df_promedio.set_index('ID', inplace=True)

# Mostrar el DataFrame resultante
print(df_promedio)

         Promedio
ID               
169484  26.833333
169523  55.333333
169524  32.800000
169529  17.777778
178223  94.000000


In [99]:
df_promedio.head()

,Promedio
ID,
169484,26.833333
169523,55.333333
169524,32.800000
169529,17.777778
178223,94.000000


In [100]:
df_promedio.to_csv('.csv', sep=';')

In [101]:
promedio_por_estacion.head()

ID
169484    26.833333
169523    55.333333
169524    32.800000
169529    17.777778
178223    94.000000
Name: Duración, dtype: float64

Por Ciclo de Cultivo Considerando los días secos en general, es decir, conjunto con todas las estaciones

In [102]:
# Calcular la duración total de los días secos por año
duracion_total_por_anio = conteo_dias_secos.groupby('Year')['Count_dias_secos'].sum()

# Contar el número total de días secos por año
num_periodos_secos_por_anio = conteo_dias_secos.groupby('Year')['Count_dias_secos'].count()

# Calcular la duración promedio de los períodos secos por año
duracion_promedio_por_anio = duracion_total_por_anio / num_periodos_secos_por_anio

# Mostrar el resultado
print("Duración promedio de los días secos por año:")
print(duracion_promedio_por_anio)


Duración promedio de los días secos por año:
Year
2024    22.8
Name: Count_dias_secos, dtype: float64


Por mes comparando entre los meses de todas las estaciones y generando un valor promedio en conjunto para todas las estaciones por mes


este debería tomar todos los nov de todas las estaciones y a partir de hay sacar un promedio por mes en general.

In [103]:
# Calcular la duración promedio de los días secos por mes y año
duracion_promedio_por_mes = resultado_periodo_seco_mes.groupby(['ID', 'Year', 'Mes'])['Duración periodo seco'].mean()

# Mostrar el resultado
print("Duración promedio de los días secos por mes:")
print(duracion_promedio_por_mes)


Duración promedio de los días secos por mes:
ID      Year  Mes
169484  2023  11     16.0
              12     26.0
        2024  1      28.0
              2      29.0
              3      15.0
              4      13.0
              5      12.0
169523  2023  11     17.0
              12     29.0
        2024  1      31.0
              2      29.0
              3      31.0
              4      28.0
              5       8.5
169524  2023  11     17.0
              12     12.5
        2024  1      27.0
              2      29.0
              3      31.0
              4      29.0
              5      10.5
169529  2023  11     15.0
              12     12.5
        2024  1      27.0
              2      14.0
              3      31.0
              4      23.0
              5       8.0
178223  2024  1       6.0
              2      29.0
              3      31.0
              4      28.0
              5      11.5
Name: Duración periodo seco, dtype: float64


**Número de días húmedos Por Ciclo de Cultivo Por Estación**
Total number of rainy days during the maize production campaign (September to April). 
A rainy day occurs when the amount of daily precipitation falls above 1 mm.

In [104]:
dia_humedo_ciclo_cultivo = df_dia_humedo

In [105]:
dia_humedo_ciclo_cultivo.info()
dia_humedo_ciclo_cultivo.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   ID           900 non-null    int64         
 1   Dia          900 non-null    period[D]     
 2   Lluvia - mm  900 non-null    float64       
 3   Date & Time  900 non-null    datetime64[ns]
 4   dia_seco     900 non-null    int64         
 5   Year         900 non-null    int32         
 6   Mes          900 non-null    int32         
dtypes: datetime64[ns](1), float64(1), int32(2), int64(2), period[D](1)
memory usage: 42.3 KB


,ID,Dia,Lluvia - mm,Date & Time,dia_seco,Year,Mes
0,169484,2023-11-14,2.6,2023-11-14,0,2023,11
1,169484,2023-11-15,0.0,2023-11-15,1,2023,11
2,169484,2023-11-16,0.8,2023-11-16,1,2023,11
3,169484,2023-11-17,0.0,2023-11-17,1,2023,11
4,169484,2023-11-18,0.0,2023-11-18,1,2023,11


In [106]:
#se elimina columnas, ya que en el df original no las tiene y en caso sea separado el desarollo vendrá sin esas columnas
dia_humedo_ciclo_cultivo = dia_humedo_ciclo_cultivo.drop(columns=['Year', 'Mes'])

In [107]:
# Filtrar el DataFrame para incluir solo los meses de mayo a septiembre
dia_humedo_filtrado = dia_humedo_ciclo_cultivo[dia_humedo_ciclo_cultivo['Date & Time'].dt.month.isin([5, 6, 7, 8, 9])]

# Crear una columna de año
dia_humedo_filtrado['Year'] = dia_humedo_filtrado['Date & Time'].dt.year

# Agrupar por año e ID y contar los días secos
conteo_dias_humedos = dia_humedo_filtrado[dia_humedo_filtrado['dia_seco'] == 0].groupby(['Year', 'ID']).size().reset_index(name='Count_dias_humedos')


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dia_humedo_filtrado['Year'] = dia_humedo_filtrado['Date & Time'].dt.year


In [108]:
conteo_dias_humedos.head()

,Year,ID,Count_dias_humedos
0,2024,169484,1
1,2024,169523,5
2,2024,169524,3
3,2024,169529,3
4,2024,178223,2


In [109]:
conteo_dias_humedos.to_csv('.csv', index=False, sep=';')

**No. de días húmedos Por Mes Por Estación**
A rainy day occurs when the amount of daily precipitation falls above 1 mm.

In [110]:
dia_humedo_por_mes = df_dia_humedo

In [111]:
dia_humedo_por_mes.head()

,ID,Dia,Lluvia - mm,Date & Time,dia_seco,Year,Mes
0,169484,2023-11-14,2.6,2023-11-14,0,2023,11
1,169484,2023-11-15,0.0,2023-11-15,1,2023,11
2,169484,2023-11-16,0.8,2023-11-16,1,2023,11
3,169484,2023-11-17,0.0,2023-11-17,1,2023,11
4,169484,2023-11-18,0.0,2023-11-18,1,2023,11


In [112]:
# Crear columnas de año y mes
#dia_humedo_por_mes['Year'] = dia_humedo_por_mes['Date & Time'].dt.year
#dia_humedo_por_mes['Mes'] = dia_humedo_por_mes['Date & Time'].dt.month

# Agrupar por año, mes e ID y contar los días secos
conteo_dias_humedos_por_mes = dia_humedo_por_mes[dia_humedo_por_mes['dia_seco'] == 0].groupby(['Year', 'Mes', 'ID']).size().reset_index(name='Count_dias_humedos_por_mes')

In [113]:
conteo_dias_humedos_por_mes.head()

,Year,Mes,ID,Count_dias_humedos_por_mes
0,2023,11,169484,1
1,2023,11,169529,1
2,2023,12,169484,1
3,2023,12,169523,1
4,2023,12,169524,3


In [114]:
conteo_dias_humedos_por_mes.to_csv('.csv', index=False, sep=';')

**Número de períodos húmedos Por Ciclo de Cultivo Por Estación**
Total number of rainy periods during the maize production campaign (September to April). 
A wet period is defined as a period of five or more consecutive days of rain followed immediately by a dry day.

In [115]:
periodo_humedo = dia_humedo_por_mes

In [116]:
periodo_humedo.info()
periodo_humedo.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   ID           900 non-null    int64         
 1   Dia          900 non-null    period[D]     
 2   Lluvia - mm  900 non-null    float64       
 3   Date & Time  900 non-null    datetime64[ns]
 4   dia_seco     900 non-null    int64         
 5   Year         900 non-null    int32         
 6   Mes          900 non-null    int32         
dtypes: datetime64[ns](1), float64(1), int32(2), int64(2), period[D](1)
memory usage: 42.3 KB


,ID,Dia,Lluvia - mm,Date & Time,dia_seco,Year,Mes
0,169484,2023-11-14,2.6,2023-11-14,0,2023,11
1,169484,2023-11-15,0.0,2023-11-15,1,2023,11
2,169484,2023-11-16,0.8,2023-11-16,1,2023,11
3,169484,2023-11-17,0.0,2023-11-17,1,2023,11
4,169484,2023-11-18,0.0,2023-11-18,1,2023,11


In [117]:
# Obtener el año más reciente en el DataFrame
año_presente = periodo_seco['Year'].max()
año_anterior = año_presente - 1

# Filtrar por los meses de septiembre del año anterior a abril del año presente
filtro_mes = ((periodo_humedo['Year'] == año_anterior) & (periodo_humedo['Mes'] >= 9)) | ((periodo_humedo['Year'] == año_presente) & (periodo_humedo['Mes'] <= 4))
df_filtrado_mes_humedo = periodo_humedo[filtro_mes]

In [118]:
df_duracion_promedio_humedo = df_filtrado_mes_humedo

In [119]:
df_filtrado_mes_humedo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 772 entries, 0 to 871
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   ID           772 non-null    int64         
 1   Dia          772 non-null    period[D]     
 2   Lluvia - mm  772 non-null    float64       
 3   Date & Time  772 non-null    datetime64[ns]
 4   dia_seco     772 non-null    int64         
 5   Year         772 non-null    int32         
 6   Mes          772 non-null    int32         
dtypes: datetime64[ns](1), float64(1), int32(2), int64(2), period[D](1)
memory usage: 42.2 KB


In [120]:
# Ordenar el DataFrame por 'ID' y 'Date & Time'
df_filtrado_mes_humedo = df_filtrado_mes_humedo.sort_values(by=['ID', 'Date & Time'])

# Inicializar un DataFrame para guardar los resultados
resultado_periodo_humedo = pd.DataFrame(columns=['ID', 'No. de periodos húmedos'])

# Inicializar variables
prev_id = None
dias_con_lluvia = 0
num_periodos_humedos = 0

# Lista temporal para almacenar resultados
resultados_temp = []

# Iterar sobre las filas del DataFrame
for index, row in df_filtrado_mes_humedo.iterrows():
    current_id = row['ID']
    
    if current_id != prev_id and prev_id is not None:
        # Guardar el conteo de periodos húmedos para el ID anterior
        resultados_temp.append({'ID': prev_id, 'No. de periodos húmedos': num_periodos_humedos})
        # Reiniciar el contador de periodos húmedos
        num_periodos_humedos = 0
        dias_con_lluvia = 0
    
    if row['dia_seco'] == 0:
        dias_con_lluvia += 1
    else:
        if dias_con_lluvia >= 5:
            num_periodos_humedos += 1
        dias_con_lluvia = 0
    
    prev_id = current_id

# Agregar el conteo para el último ID
if prev_id is not None:
    resultados_temp.append({'ID': prev_id, 'No. de periodos húmedos': num_periodos_humedos})

# Crear el DataFrame final con los resultados
resultado_periodo_humedo = pd.DataFrame(resultados_temp)

# Convertir el ID a int64
resultado_periodo_humedo['ID'] = resultado_periodo_humedo['ID'].astype('int64')

# Mostrar el DataFrame resultado
print(resultado_periodo_humedo)

       ID  No. de periodos húmedos
0  169484                        0
1  169523                        0
2  169524                        0
3  169529                        0
4  178223                        0


In [121]:
resultado_periodo_humedo.head()

,ID,No. de periodos húmedos
0,169484,0
1,169523,0
2,169524,0
3,169529,0
4,178223,0


In [122]:
resultado_periodo_humedo.to_csv('.csv', index=False, sep=';')

**No de períodos húmedos Por Mes por Estación**

In [123]:
periodo_humedo_mes = periodo_humedo

In [124]:
# Verificar que el DataFrame contiene las columnas necesarias
required_columns = ['ID', 'Year', 'Mes', 'Date & Time', 'dia_seco']
for col in required_columns:
    if col not in periodo_humedo_mes.columns:
        raise ValueError(f"Falta la columna necesaria: {col}")

# Ordenar el DataFrame por 'ID', 'Year', 'Mes' y 'Date & Time'
df_orden_mes = periodo_humedo_mes.sort_values(by=['ID', 'Year', 'Mes', 'Date & Time'])

# Inicializar un DataFrame para guardar los resultados
resultado_periodo_humedo_mes = pd.DataFrame(columns=['ID', 'Year', 'Mes', 'Inicio periodo humedo', 'Fin periodo humedo', 'Duración periodo humedo', 'Dias con lluvia', 'periodo_humedo_mes'])

# Inicializar variables
prev_id = None
prev_year = None
prev_mes = None
dias_con_lluvia = 0
inicio_periodo = None

# Lista temporal para almacenar resultados
resultados_temp = []

# Iterar sobre las filas del DataFrame
for index, row in df_orden_mes.iterrows():
    current_id = row['ID']
    current_year = row['Year']
    current_mes = row['Mes']
    current_date = row['Date & Time']
    
    if (current_id != prev_id or current_year != prev_year or current_mes != prev_mes) and prev_id is not None:
        # Guardar el conteo de periodos húmedos para el ID, año y mes anterior
        if dias_con_lluvia > 0:
            resultados_temp.append({
                'ID': prev_id, 
                'Year': prev_year, 
                'Mes': prev_mes, 
                'Inicio periodo humedo': inicio_periodo,
                'Fin periodo humedo': prev_date,
                'Duración periodo humedo': dias_con_lluvia,
                'Dias con lluvia': dias_con_lluvia,
                'periodo_humedo_mes': 1 if dias_con_lluvia >= 5 else 0
            })
        # Reiniciar el contador de periodos húmedos y días con lluvia
        dias_con_lluvia = 0
        inicio_periodo = None
    
    if row['dia_seco'] == 0:
        if dias_con_lluvia == 0:
            inicio_periodo = current_date
        dias_con_lluvia += 1
    else:
        if dias_con_lluvia > 0:
            resultados_temp.append({
                'ID': current_id, 
                'Year': current_year, 
                'Mes': current_mes, 
                'Inicio periodo humedo': inicio_periodo,
                'Fin periodo humedo': current_date - pd.Timedelta(days=1),
                'Duración periodo humedo': dias_con_lluvia,
                'Dias con lluvia': dias_con_lluvia,
                'periodo_humedo_mes': 1 if dias_con_lluvia >= 5 else 0
            })
        dias_con_lluvia = 0
        inicio_periodo = None
    
    prev_id = current_id
    prev_year = current_year
    prev_mes = current_mes
    prev_date = current_date

# Agregar el conteo para el último ID, año y mes
if prev_id is not None and dias_con_lluvia > 0:
    resultados_temp.append({
        'ID': prev_id, 
        'Year': prev_year, 
        'Mes': prev_mes, 
        'Inicio periodo humedo': inicio_periodo,
        'Fin periodo humedo': prev_date,
        'Duración periodo humedo': dias_con_lluvia,
        'Dias con lluvia': dias_con_lluvia,
        'periodo_humedo_mes': 1 if dias_con_lluvia >= 5 else 0
    })

# Crear el DataFrame final con los resultados
resultado_periodo_humedo_mes = pd.DataFrame(resultados_temp)

# Convertir el ID a int64
resultado_periodo_humedo_mes['ID'] = resultado_periodo_humedo_mes['ID'].astype('int64')

In [125]:
resultado_periodo_humedo_mes.info()
resultado_periodo_humedo_mes.head(15)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   ID                       31 non-null     int64         
 1   Year                     31 non-null     int64         
 2   Mes                      31 non-null     int64         
 3   Inicio periodo humedo    31 non-null     datetime64[ns]
 4   Fin periodo humedo       31 non-null     datetime64[ns]
 5   Duración periodo humedo  31 non-null     int64         
 6   Dias con lluvia          31 non-null     int64         
 7   periodo_humedo_mes       31 non-null     int64         
dtypes: datetime64[ns](2), int64(6)
memory usage: 2.1 KB


,ID,Year,Mes,Inicio periodo humedo,Fin periodo humedo,Duración periodo humedo,Dias con lluvia,periodo_humedo_mes
0,169484,2023,11,2023-11-14,2023-11-14,1,1,0
1,169484,2023,12,2023-12-05,2023-12-05,1,1,0
2,169484,2024,1,2024-01-29,2024-01-29,1,1,0
3,169484,2024,3,2024-03-18,2024-03-18,1,1,0
4,169484,2024,4,2024-04-08,2024-04-09,2,2,0
5,169484,2024,4,2024-04-29,2024-04-30,2,2,0
6,169484,2024,5,2024-05-16,2024-05-16,1,1,0
7,169523,2023,12,2023-12-30,2023-12-30,1,1,0
8,169523,2024,4,2024-04-29,2024-04-30,2,2,0
9,169523,2024,5,2024-05-11,2024-05-11,1,1,0


In [126]:
# Agrupar por ID, Year y Mes y sumar la columna periodo_seco_mes
cantidad_periodo_humedo_mes = resultado_periodo_humedo_mes.groupby(['ID', 'Year', 'Mes']).agg({'periodo_humedo_mes': 'sum'}).reset_index()

# Renombrar la columna periodo_seco_mes a Cantidad de periodos secos
cantidad_periodo_humedo_mes = cantidad_periodo_humedo_mes.rename(columns={'periodo_humedo_mes': 'Cantidad de periodos humedos'})

# Convertir el ID a int64
cantidad_periodo_humedo_mes['ID'] = cantidad_periodo_humedo_mes['ID'].astype('int64')

# Mostrar el DataFrame resultado
print(cantidad_periodo_humedo_mes)

        ID  Year  Mes  Cantidad de periodos humedos
0   169484  2023   11                             0
1   169484  2023   12                             0
2   169484  2024    1                             0
3   169484  2024    3                             0
4   169484  2024    4                             0
5   169484  2024    5                             0
6   169523  2023   12                             0
7   169523  2024    4                             0
8   169523  2024    5                             0
9   169524  2023   12                             0
10  169524  2024    1                             0
11  169524  2024    4                             0
12  169524  2024    5                             0
13  169529  2023   11                             0
14  169529  2023   12                             0
15  169529  2024    1                             0
16  169529  2024    2                             0
17  169529  2024    4                             0
18  169529  

**Duración promedio de los períodos húmedos**
Average duration of rainy periods during the maize production campaign (September to April). This index describes the duration in days of the rainy periods.

Por Ciclo de cultivo para cada estación: 
Ej:
ID 169484 tiene 3 periodos húmedos

|    periodo    | Duración(días)|
| ------------- | ------------- |
| 1             |20             |
| 2             |54             |
| 3             |48             |

  

Duración promedio de los períodos secos: 40.67 días

In [127]:
df_duracion_promedio_humedo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 772 entries, 0 to 871
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   ID           772 non-null    int64         
 1   Dia          772 non-null    period[D]     
 2   Lluvia - mm  772 non-null    float64       
 3   Date & Time  772 non-null    datetime64[ns]
 4   dia_seco     772 non-null    int64         
 5   Year         772 non-null    int32         
 6   Mes          772 non-null    int32         
dtypes: datetime64[ns](1), float64(1), int32(2), int64(2), period[D](1)
memory usage: 42.2 KB


In [128]:
df_duracion_promedio_humedo.head()

,ID,Dia,Lluvia - mm,Date & Time,dia_seco,Year,Mes
0,169484,2023-11-14,2.6,2023-11-14,0,2023,11
1,169484,2023-11-15,0.0,2023-11-15,1,2023,11
2,169484,2023-11-16,0.8,2023-11-16,1,2023,11
3,169484,2023-11-17,0.0,2023-11-17,1,2023,11
4,169484,2023-11-18,0.0,2023-11-18,1,2023,11


In [129]:
# Ordenar el DataFrame por 'ID' y 'Date & Time'
df_duracion_promedio_humedo = df_duracion_promedio_humedo.sort_values(by=['ID', 'Date & Time'])

# Inicializar listas para guardar los resultados
ids = []
inicio_rango = []
fin_rango = []
duracion = []

# Inicializar variables
dias_con_lluvia = 0
inicio_con_lluvia = None
prev_id = None

# Iterar sobre las filas del DataFrame
for index, row in df_duracion_promedio_humedo.iterrows():
    current_id = row['ID']
    dia_seco = row['dia_seco']
    
    if current_id != prev_id:
        # Guardar el rango de días con lluvia para el ID anterior
        if prev_id is not None and inicio_con_lluvia is not None:
            ids.append(prev_id)
            inicio_rango.append(inicio_con_lluvia)
            fin_rango.append(prev_fecha)
            duracion.append(dias_con_lluvia)
        
        # Reiniciar variables al cambiar de ID
        dias_con_lluvia = 0
        inicio_con_lluvia = None
    
    if dia_seco == 0:
        if inicio_con_lluvia is None:
            inicio_con_lluvia = row['Date & Time']
        dias_con_lluvia += 1
        prev_fecha = row['Date & Time']
    else:
        if inicio_con_lluvia is not None:
            # Guardar el rango de días con lluvia cuando se encuentra un día seco
            ids.append(prev_id)
            inicio_rango.append(inicio_con_lluvia)
            fin_rango.append(prev_fecha)
            duracion.append(dias_con_lluvia)
            # Reiniciar variables para el nuevo período de días con lluvia
            dias_con_lluvia = 0
            inicio_con_lluvia = None
    
    prev_id = current_id

# Agregar el rango de días con lluvia para el último ID
if prev_id is not None and inicio_con_lluvia is not None:
    ids.append(prev_id)
    inicio_rango.append(inicio_con_lluvia)
    fin_rango.append(prev_fecha)
    duracion.append(dias_con_lluvia)

# Convertir las listas a un DataFrame
resultado_dias_con_lluvia = pd.DataFrame({
    'ID': ids,
    'Inicio del rango': inicio_rango,
    'Fin del rango': fin_rango,
    'Duración': duracion
})

# Convertir el ID a int64
resultado_dias_con_lluvia['ID'] = resultado_dias_con_lluvia['ID'].astype('int64')

# Mostrar el DataFrame resultado
print(resultado_dias_con_lluvia)


        ID Inicio del rango Fin del rango  Duración
0   169484       2023-11-14    2023-11-14         1
1   169484       2023-12-05    2023-12-05         1
2   169484       2024-01-29    2024-01-29         1
3   169484       2024-03-18    2024-03-18         1
4   169484       2024-04-08    2024-04-09         2
5   169484       2024-04-29    2024-04-30         2
6   169523       2023-12-30    2023-12-30         1
7   169523       2024-04-29    2024-04-30         2
8   169524       2023-12-04    2023-12-05         2
9   169524       2023-12-11    2023-12-11         1
10  169524       2024-01-28    2024-01-28         1
11  169524       2024-04-30    2024-04-30         1
12  169529       2023-11-15    2023-11-15         1
13  169529       2023-12-04    2023-12-05         2
14  169529       2023-12-11    2023-12-11         1
15  169529       2024-01-28    2024-01-28         1
16  169529       2024-02-18    2024-02-18         1
17  169529       2024-04-03    2024-04-03         1
18  169529  

In [130]:

# Convertir las columnas de fechas a tipo datetime
resultado_dias_con_lluvia['Inicio del rango'] = pd.to_datetime(resultado_dias_con_lluvia['Inicio del rango'])
resultado_dias_con_lluvia['Fin del rango'] = pd.to_datetime(resultado_dias_con_lluvia['Fin del rango'])

# Eliminar los registros con NaT en la columna 'Inicio del rango'
resultado_dias_con_lluvia = resultado_dias_con_lluvia.dropna(subset=['Inicio del rango'])

# Filtrar los registros que tengan Duración mayor a 5
filtrado_dias_con_lluvia = resultado_dias_con_lluvia[resultado_dias_con_lluvia['Duración'] > 5]

# Mostrar el DataFrame resultado
print(filtrado_dias_con_lluvia)



Empty DataFrame
Columns: [ID, Inicio del rango, Fin del rango, Duración]
Index: []


In [131]:
###hay que eliminar antes el 0 de duración
# Calcular el promedio de duración por estación
promedio_por_estacion_humedo = filtrado_dias_con_lluvia.groupby('ID')['Duración'].mean()

# Mostrar el resultado
print("Promedio de duración por estación:")
print(promedio_por_estacion_humedo)


Promedio de duración por estación:
Series([], Name: Duración, dtype: float64)


In [132]:
# Convertir el diccionario en DataFrame
df_promedio_humedo = pd.DataFrame(list(promedio_por_estacion_humedo.items()), columns=['ID', 'Promedio'])

# Establecer 'ID' como índice
df_promedio_humedo.set_index('ID', inplace=True)

# Mostrar el DataFrame resultante
print(df_promedio_humedo)


Empty DataFrame
Columns: [Promedio]
Index: []


In [133]:
df_promedio_humedo.head()

,Promedio
ID,


**Número de días calurosos**
Total number of hot days during the maize production campaign (Mayo - Septiembrel). This index is described as days with maximum temperatures above 33°C (critical temperatures that affect the growth and development of corn).

Importancia:

Impacto en el Crecimiento del Maíz:

Las temperaturas superiores a 33°C pueden provocar estrés térmico en las plantas de maíz.
Este estrés puede reducir la fotosíntesis, lo que a su vez disminuye el crecimiento de la planta y la producción de granos.
Además, las temperaturas muy altas pueden afectar la polinización y el llenado de granos, resultando en una menor productividad.
Planificación Agrícola:

Conocer el número de días calurosos ayuda a los agricultores y agrónomos a tomar decisiones informadas sobre el manejo de cultivos.
Puede influir en la elección de variedades de maíz más resistentes al calor.
Ayuda a planificar estrategias de riego y manejo del suelo para mitigar el impacto del calor extremo.

In [134]:
df_data_stations_day_hot = df_data_all_stations

In [135]:
df_data_stations_day_hot.info()
df_data_stations_day_hot.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21434 entries, 0 to 21433
Data columns (total 27 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   Date & Time                       21434 non-null  datetime64[ns]
 1   ID                                21434 non-null  int64         
 2   Alta velocidad del viento - km/h  21433 non-null  float64       
 3   BarÃ³metro - mb                   18449 non-null  float64       
 4   Bulbo HÃºmedo - Â°C               18429 non-null  float64       
 5   DÃ­as-grado de calentamiento      18429 non-null  float64       
 6   DÃ­as-grado de enfriamiento       18429 non-null  float64       
 7   ET - mm                           21433 non-null  float64       
 8   EnergÃ­a solar - Ly               18429 non-null  float64       
 9   Hum - %                           19690 non-null  float64       
 10  Humedad del suelo  cm - cb        18408 non-nu

,Date & Time,ID,Alta velocidad del viento - km/h,BarÃ³metro - mb,Bulbo HÃºmedo - Â°C,DÃ­as-grado de calentamiento,DÃ­as-grado de enfriamiento,ET - mm,EnergÃ­a solar - Ly,Hum - %,...,THSW Ã­ndice - Â°C,THW Index - Â°C,Tasa de lluvia - mm/h,Temp - Â°C,Temp. Baja - Â°C,Tempalta,Velocidad del viento - km/h,Viento Corriente - km,Viento FrÃ­o - Â°C,Ãndice de calor - Â°C
0,2024-01-01 00:00:00,169484,4.8,1025.0,14.9,0.037,0.0,0.00,0.0,77.0,...,16.7,17.5,0.0,17.4,17.4,17.7,1.6,1.61,17.4,17.5
1,2024-01-01 00:00:00,169523,3.2,972.9,16.2,0.012,0.0,0.00,0.0,84.0,...,17.2,18.4,0.0,18.1,17.8,18.3,0.0,0.00,18.1,18.4
2,2024-01-01 00:00:00,169524,3.2,1023.6,15.0,0.021,0.0,0.00,0.0,75.0,...,16.7,17.8,0.0,17.8,17.7,17.9,0.0,0.00,17.8,17.8
3,2024-01-01 00:00:00,169529,8.0,946.3,14.4,0.030,0.0,0.03,0.0,73.0,...,16.7,17.5,0.0,17.6,17.4,17.8,3.2,3.22,17.6,17.5
4,2024-01-01 01:00:00,169484,4.8,1024.6,15.1,0.044,0.0,0.00,0.0,80.0,...,16.1,17.4,0.0,17.3,17.0,17.6,1.6,1.61,17.3,17.4


In [136]:
# Convertir 'Date & Time' a formato datetime manejando diferentes formatos de fecha
df_data_stations_day_hot['Date & Time'] = pd.to_datetime(df_data_stations_day_hot['Date & Time'], errors='coerce', dayfirst=True)

# Filtrar el DataFrame para conservar solo las columnas necesarias
df_temps = df_data_stations_day_hot[['Date & Time', 'ID', 'Temp - Â°C','Temp. Baja - Â°C','Tempalta']].copy()

In [137]:
df_temps.info()
df_temps.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21434 entries, 0 to 21433
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date & Time       21434 non-null  datetime64[ns]
 1   ID                21434 non-null  int64         
 2   Temp - Â°C        19405 non-null  float64       
 3   Temp. Baja - Â°C  19405 non-null  float64       
 4   Tempalta          19405 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1)
memory usage: 837.4 KB


,Date & Time,ID,Temp - Â°C,Temp. Baja - Â°C,Tempalta
0,2024-01-01 00:00:00,169484,17.4,17.4,17.7
1,2024-01-01 00:00:00,169523,18.1,17.8,18.3
2,2024-01-01 00:00:00,169524,17.8,17.7,17.9
3,2024-01-01 00:00:00,169529,17.6,17.4,17.8
4,2024-01-01 01:00:00,169484,17.3,17.0,17.6


Día caluroso = 1 (sí)
Día No caluroso = 0(no)

In [138]:
# Crear la nueva columna 'hot_days'
df_temps['hot_days'] = df_temps['Tempalta'].apply(lambda x: 1 if x >= 30 else 0)

# Mostrar el DataFrame resultado
print(df_temps)


              Date & Time      ID  Temp - Â°C  Temp. Baja - Â°C  Tempalta  \
0     2024-01-01 00:00:00  169484        17.4              17.4      17.7   
1     2024-01-01 00:00:00  169523        18.1              17.8      18.3   
2     2024-01-01 00:00:00  169524        17.8              17.7      17.9   
3     2024-01-01 00:00:00  169529        17.6              17.4      17.8   
4     2024-01-01 01:00:00  169484        17.3              17.0      17.6   
...                   ...     ...         ...               ...       ...   
21429 2023-12-09 22:00:00  169529        21.2              20.9      21.3   
21430 2023-12-09 23:00:00  169484        18.3              17.9      18.9   
21431 2023-12-09 23:00:00  169523        20.5              19.9      20.8   
21432 2023-12-09 23:00:00  169524        16.3              15.9      17.0   
21433 2023-12-09 23:00:00  169529        20.7              20.5      20.9   

       hot_days  
0             0  
1             0  
2             0  
3  

In [139]:
# Crear el nuevo DataFrame filtrando los días calurosos
dias_calurosos = df_temps[df_temps['hot_days'] == 1]

In [140]:
dias_calurosos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2896 entries, 283 to 21404
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date & Time       2896 non-null   datetime64[ns]
 1   ID                2896 non-null   int64         
 2   Temp - Â°C        2896 non-null   float64       
 3   Temp. Baja - Â°C  2896 non-null   float64       
 4   Tempalta          2896 non-null   float64       
 5   hot_days          2896 non-null   int64         
dtypes: datetime64[ns](1), float64(3), int64(2)
memory usage: 158.4 KB


In [141]:
# Filtrar el DataFrame para incluir solo los meses de mayo a septiembre
df_dias_calurosos_filtrado = dias_calurosos[dias_calurosos['Date & Time'].dt.month.isin([5, 6, 7, 8, 9])]

# Crear una columna de año
df_dias_calurosos_filtrado['Year'] = df_dias_calurosos_filtrado['Date & Time'].dt.year

# Agrupar por año e ID (suponiendo que existe una columna 'ID') y contar los días calurosos

conteo_dias_calurosos = df_dias_calurosos_filtrado[df_dias_calurosos_filtrado['hot_days'] == 1].groupby(['Year', 'ID']).size().reset_index(name='Conteo_dias_calurosos')


# Mostrar el DataFrame resultado
print(conteo_dias_calurosos)

   Year      ID  Conteo_dias_calurosos
0  2024  169484                    199
1  2024  169523                    210
2  2024  169524                    228
3  2024  169529                    191


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dias_calurosos_filtrado['Year'] = df_dias_calurosos_filtrado['Date & Time'].dt.year


In [142]:
df_dias_calurosos_filtrado.to_csv('.csv', index=False, sep=';')

In [143]:
conteo_dias_calurosos.head()

,Year,ID,Conteo_dias_calurosos
0,2024,169484,199
1,2024,169523,210
2,2024,169524,228
3,2024,169529,191


**Días Calurosos Por Mes Por Estación**

In [144]:
dias_calurosos_mes = dias_calurosos

In [145]:
# Extraer el mes y el año de la columna 'Date & Time'
dias_calurosos_mes['Year'] = dias_calurosos_mes['Date & Time'].dt.year
dias_calurosos_mes['Month'] = dias_calurosos_mes['Date & Time'].dt.month

# Agrupar por año, mes e ID y contar los días calurosos
conteo_dias_calurosos_mes = dias_calurosos_mes.groupby(['Year', 'Month', 'ID']).size().reset_index(name='Conteo_dias_calurosos_mes')

# Mostrar el DataFrame resultado
print(conteo_dias_calurosos_mes)

    Year  Month      ID  Conteo_dias_calurosos_mes
0   2023     11  169484                          2
1   2023     11  169523                          2
2   2023     11  169524                         16
3   2023     12  169484                         10
4   2023     12  169523                         19
5   2023     12  169524                         43
6   2023     12  169529                          3
7   2024      1  169484                         28
8   2024      1  169523                         51
9   2024      1  169524                         95
10  2024      1  169529                         40
11  2024      2  169484                         53
12  2024      2  169523                         72
13  2024      2  169524                        101
14  2024      2  169529                         63
15  2024      3  169484                        165
16  2024      3  169523                        216
17  2024      3  169524                        223
18  2024      3  169529        

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dias_calurosos_mes['Year'] = dias_calurosos_mes['Date & Time'].dt.year
: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dias_calurosos_mes['Month'] = dias_calurosos_mes['Date & Time'].dt.month


**Número de olas de calor**
Total number of heat waves during the maize production season (Mayo - Septiembre). This index consists of three or more consecutive heat days and each heat wave must be separated from one another by at least one heatless day.


1- Tres o más días consecutivos de calor: Para que se considere una ola de calor, debe haber al menos tres días seguidos con temperaturas altas que cumplen con un umbral específico de calor (generalmente definido en términos de temperatura mínima o máxima).

2- Separación por al menos un día sin calor: Para contar olas de calor separadas, debe haber al menos un día entre ellas que no alcance el umbral de temperatura alta. Es decir, dos olas de calor consecutivas no pueden estar adyacentes; debe haber un día intermedio sin condiciones de calor extremo para que se consideren como dos eventos distintos.

Este índice es importante para la agricultura y el manejo de cultivos porque las olas de calor pueden tener un impacto significativo en el crecimiento y la salud de los cultivos de maíz. Monitorear y entender la frecuencia y la duración de las olas de calor ayuda a los agricultores y planificadores a tomar decisiones informadas sobre riego, protección de cultivos y otras prácticas de manejo agrícola para mitigar los efectos adversos del calor extremo.

In [146]:
df_ola_calor = df_temps

In [147]:
# Extraer el mes y el año de la columna 'Date & Time'
df_ola_calor['Year'] = df_ola_calor['Date & Time'].dt.year
df_ola_calor['Month'] = df_ola_calor['Date & Time'].dt.month

In [148]:
df_ola_calor.info()
df_ola_calor.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21434 entries, 0 to 21433
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date & Time       21434 non-null  datetime64[ns]
 1   ID                21434 non-null  int64         
 2   Temp - Â°C        19405 non-null  float64       
 3   Temp. Baja - Â°C  19405 non-null  float64       
 4   Tempalta          19405 non-null  float64       
 5   hot_days          21434 non-null  int64         
 6   Year              21434 non-null  int32         
 7   Month             21434 non-null  int32         
dtypes: datetime64[ns](1), float64(3), int32(2), int64(2)
memory usage: 1.1 MB


,Date & Time,ID,Temp - Â°C,Temp. Baja - Â°C,Tempalta,hot_days,Year,Month
0,2024-01-01 00:00:00,169484,17.4,17.4,17.7,0,2024,1
1,2024-01-01 00:00:00,169523,18.1,17.8,18.3,0,2024,1
2,2024-01-01 00:00:00,169524,17.8,17.7,17.9,0,2024,1
3,2024-01-01 00:00:00,169529,17.6,17.4,17.8,0,2024,1
4,2024-01-01 01:00:00,169484,17.3,17.0,17.6,0,2024,1


In [149]:
# Obtener el año más reciente en el DataFrame
año_presente = df_ola_calor['Year'].max()
año_anterior = año_presente - 1

# Filtrar por los meses de septiembre del año anterior a abril del año presente
filtro_meses = ((df_ola_calor['Year'] == año_anterior) & (df_ola_calor['Month'] >= 9)) | ((df_ola_calor['Year'] == año_presente) & (df_ola_calor['Month'] <= 4))
df_ola_calor_cultivo = df_ola_calor[filtro_meses]

In [150]:
df_ola_calor_cultivo.to_csv('.csv', index=False, sep=';')

In [151]:


# Ordenar el DataFrame por 'ID' y 'Date & Time'
df_ola_calor_cultivo = df_ola_calor_cultivo.sort_values(by=['ID', 'Date & Time'])

# Inicializar un DataFrame para guardar los resultados
resultado_periodo_seco = pd.DataFrame(columns=['ID', 'No. de Olas de Calor'])

# Inicializar variables
prev_id = None
dias_sin_lluvia = 0
num_periodos_secos = 0

# Lista temporal para almacenar resultados
resultados_temp = []

# Iterar sobre las filas del DataFrame
for index, row in df_ola_calor_cultivo.iterrows():
    current_id = row['ID']
    
    if current_id != prev_id and prev_id is not None:
        # Guardar el conteo de periodos secos para el ID anterior
        resultados_temp.append({'ID': prev_id, 'No. de Olas de Calor': num_periodos_secos})
        # Reiniciar el contador de periodos secos
        num_periodos_secos = 0
        dias_sin_lluvia = 0
    
    if row['hot_days'] == 1:
        dias_sin_lluvia += 1
    else:
        if dias_sin_lluvia >= 3:
            num_periodos_secos += 1
        dias_sin_lluvia = 0
    
    prev_id = current_id

# Agregar el conteo para el último ID
if prev_id is not None:
    resultados_temp.append({'ID': prev_id, 'No. de Olas de Calor': num_periodos_secos})

# Crear el DataFrame final con los resultados
resultado_ola_calor_cultivo = pd.DataFrame(resultados_temp)

# Convertir el ID a int64
resultado_ola_calor_cultivo['ID'] = resultado_ola_calor_cultivo['ID'].astype('int64')

# Mostrar el DataFrame resultado
print(resultado_ola_calor_cultivo)


       ID  No. de Olas de Calor
0  169484                    65
1  169523                    79
2  169524                   101
3  169529                    70
4  178223                     0


Comprobación de rangos de hot_days
También, sirve para sacar la duración de las olas de calor en días

In [152]:
# Ordenar el DataFrame por 'ID' y 'Date & Time'
df_filtrado = df_ola_calor_cultivo.sort_values(by=['ID', 'Date & Time'])

# Inicializar listas para guardar los resultados
ids = []
inicio_rango = []
fin_rango = []
duracion = []

# Inicializar variables
dias_calor = 0
inicio_calor = None
prev_id = None
prev_hot_day = 0

# Iterar sobre las filas del DataFrame
for index, row in df_filtrado.iterrows():
    current_id = row['ID']
    hot_day = row['hot_days']
    
    if current_id != prev_id:
        # Guardar el rango de días de calor para el ID anterior
        if prev_id is not None and dias_calor >= 3:
            ids.append(prev_id)
            inicio_rango.append(inicio_calor)
            fin_rango.append(prev_fecha)
            duracion.append(dias_calor)
        
        # Reiniciar variables al cambiar de ID
        dias_calor = 0
        inicio_calor = None
    
    if hot_day == 1:
        if inicio_calor is None:
            inicio_calor = row['Date & Time']
        dias_calor += 1
        prev_fecha = row['Date & Time']
        prev_hot_day = 1
    else:
        if prev_hot_day == 1:
            if dias_calor >= 3:
                ids.append(prev_id)
                inicio_rango.append(inicio_calor)
                fin_rango.append(prev_fecha)
                duracion.append(dias_calor)
            dias_calor = 0
            inicio_calor = None
        prev_hot_day = 0
    
    prev_id = current_id

# Agregar el rango de días de calor para el último ID
if prev_id is not None and dias_calor >= 3:
    ids.append(prev_id)
    inicio_rango.append(inicio_calor)
    fin_rango.append(prev_fecha)
    duracion.append(dias_calor)

# Convertir las listas a un DataFrame
resultado_olas_calor = pd.DataFrame({
    'ID': ids,
    'Inicio del rango': inicio_rango,
    'Fin del rango': fin_rango,
    'Duración': duracion
})

# Convertir el ID a int64
resultado_olas_calor['ID'] = resultado_olas_calor['ID'].astype('int64')

# Mostrar el DataFrame resultado
print(resultado_olas_calor)

         ID    Inicio del rango       Fin del rango  Duración
0    169484 2023-12-01 14:00:00 2023-12-01 16:00:00         3
1    169484 2023-12-25 14:00:00 2023-12-25 17:00:00         4
2    169484 2024-01-05 15:00:00 2024-01-05 17:00:00         3
3    169484 2024-01-06 14:00:00 2024-01-06 17:00:00         4
4    169484 2024-01-09 14:00:00 2024-01-09 18:00:00         5
..      ...                 ...                 ...       ...
310  169529 2024-04-26 12:00:00 2024-04-26 17:00:00         6
311  169529 2024-04-27 12:00:00 2024-04-27 18:00:00         7
312  169529 2024-04-28 11:00:00 2024-04-28 18:00:00         8
313  169529 2024-04-29 12:00:00 2024-04-29 18:00:00         7
314  169529 2024-04-30 12:00:00 2024-04-30 15:00:00         4

[315 rows x 4 columns]


In [153]:
resultado_olas_calor.to_csv('.csv', index=False, sep=';')

Olas de Calor Por mes Por estación

In [154]:
df_ola_calor_mes = df_temps

In [155]:
df_ola_calor_mes.head()

,Date & Time,ID,Temp - Â°C,Temp. Baja - Â°C,Tempalta,hot_days,Year,Month
0,2024-01-01 00:00:00,169484,17.4,17.4,17.7,0,2024,1
1,2024-01-01 00:00:00,169523,18.1,17.8,18.3,0,2024,1
2,2024-01-01 00:00:00,169524,17.8,17.7,17.9,0,2024,1
3,2024-01-01 00:00:00,169529,17.6,17.4,17.8,0,2024,1
4,2024-01-01 01:00:00,169484,17.3,17.0,17.6,0,2024,1


In [156]:
# Verificar que el DataFrame contiene las columnas necesarias
required_columns = ['ID', 'Year', 'Month', 'Date & Time', 'hot_days']
for col in required_columns:
    if col not in df_ola_calor_mes.columns:
        raise ValueError(f"Falta la columna necesaria: {col}")

# Ordenar el DataFrame por 'ID', 'Year', 'Mes' y 'Date & Time'
df_orden_mes = df_ola_calor_mes.sort_values(by=['ID', 'Year', 'Month', 'Date & Time'])

# Inicializar un DataFrame para guardar los resultados
resultado_olas_calor_mes = pd.DataFrame(columns=['ID', 'Year', 'Month', 'Inicio ola calor', 'Fin ola calor', 'Duración ola calor', 'Dias sin lluvia', 'ola_calor_mes'])

# Inicializar variables
prev_id = None
prev_year = None
prev_mes = None
dias_sin_lluvia = 0
inicio_periodo = None

# Lista temporal para almacenar resultados
resultados_temp = []

# Iterar sobre las filas del DataFrame
for index, row in df_orden_mes.iterrows():
    current_id = row['ID']
    current_year = row['Year']
    current_mes = row['Month']
    current_date = row['Date & Time']
    
    if (current_id != prev_id or current_year != prev_year or current_mes != prev_mes) and prev_id is not None:
        # Guardar el conteo de periodos secos para el ID, año y mes anterior
        if dias_sin_lluvia >= 5:
            resultados_temp.append({
                'ID': prev_id, 
                'Year': prev_year, 
                'Month': prev_mes, 
                'Inicio ola calor': inicio_periodo,
                'Fin ola calor': prev_date,
                'Duración ola calor': dias_sin_lluvia,
                'Dias sin lluvia': dias_sin_lluvia,
                'ola_calor_mes': 1
            })
        # Reiniciar el contador de periodos secos y días sin lluvia
        dias_sin_lluvia = 0
        inicio_periodo = None
    
    if row['hot_days'] == 1:
        if dias_sin_lluvia == 0:
            inicio_periodo = current_date
        dias_sin_lluvia += 1
    else:
        if dias_sin_lluvia >= 5:
            resultados_temp.append({
                'ID': current_id, 
                'Year': current_year, 
                'Month': current_mes, 
                'Inicio ola calor': inicio_periodo,
                'Fin ola calor': current_date - pd.Timedelta(days=1),
                'Duración ola calor': dias_sin_lluvia,
                'Dias sin lluvia': dias_sin_lluvia,
                'ola_calor_mes': 1
            })
        dias_sin_lluvia = 0
        inicio_periodo = None
    
    prev_id = current_id
    prev_year = current_year
    prev_mes = current_mes
    prev_date = current_date

# Agregar el conteo para el último ID, año y mes
if prev_id is not None and dias_sin_lluvia >= 5:
    resultados_temp.append({
        'ID': prev_id, 
        'Year': prev_year, 
        'Month': prev_mes, 
        'Inicio ola calor': inicio_periodo,
        'Fin ola calor': prev_date,
        'Duración ola calor': dias_sin_lluvia,
        'Dias sin lluvia': dias_sin_lluvia,
        'ola_calor_mes': 1
    })

# Crear el DataFrame final con los resultados
resultado_olas_calor_mes = pd.DataFrame(resultados_temp)

# Añadir la columna 'periodo_seco_mes' para todos los registros donde 'Dias sin lluvia' es menor a 5
for index, row in df_orden_mes.iterrows():
    if row['hot_days'] == 1:
        dias_sin_lluvia += 1
    else:
        if dias_sin_lluvia < 5:
            resultados_temp.append({
                'ID': current_id, 
                'Year': current_year, 
                'Month': current_mes, 
                'Inicio ola calor': inicio_periodo,
                'Fin ola calor': current_date - pd.Timedelta(days=1),
                'Duración ola calor': dias_sin_lluvia,
                'Dias sin lluvia': dias_sin_lluvia,
                'ola_calor_mes': 0
            })

# Convertir el ID a int64
resultado_olas_calor_mes['ID'] = resultado_olas_calor_mes['ID'].astype('int64')

# Agrupar por ID, Year y Mes y sumar la columna periodo_seco_mes
cantidad_olas_calor_mes = resultado_olas_calor_mes.groupby(['ID', 'Year', 'Month']).agg({'ola_calor_mes': 'sum'}).reset_index()

# Renombrar la columna periodo_seco_mes a Cantidad de periodos secos
cantidad_olas_calor_mes = cantidad_olas_calor_mes.rename(columns={'ola_calor_mes': 'cantidad_olas_calor_mes'})

# Convertir el ID a int64
cantidad_olas_calor_mes['ID'] = cantidad_olas_calor_mes['ID'].astype('int64')

# Mostrar el DataFrame resultado
print(cantidad_olas_calor_mes)


        ID  Year  Month  cantidad_olas_calor_mes
0   169484  2024      1                        1
1   169484  2024      2                        6
2   169484  2024      3                       21
3   169484  2024      4                       20
4   169484  2024      5                       24
5   169523  2023     12                        2
6   169523  2024      1                        5
7   169523  2024      2                        9
8   169523  2024      3                       28
9   169523  2024      4                       25
10  169523  2024      5                       24
11  169524  2023     11                        1
12  169524  2023     12                        6
13  169524  2024      1                       11
14  169524  2024      2                       12
15  169524  2024      3                       26
16  169524  2024      4                       25
17  169524  2024      5                       24
18  169529  2024      1                        2
19  169529  2024    

In [157]:
cantidad_olas_calor_mes.to_csv('.csv', index=False, sep=';')

**Duración máxima de las olas de calor**

Maximum duration of heat waves during the maize production campaign (Mayo - Septiembre).

In [158]:
#obtenido de la comprobación de rangos de hot_days
duración_ola_calor = resultado_olas_calor

In [159]:
duración_ola_calor.info()
duración_ola_calor.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315 entries, 0 to 314
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ID                315 non-null    int64         
 1   Inicio del rango  315 non-null    datetime64[ns]
 2   Fin del rango     315 non-null    datetime64[ns]
 3   Duración          315 non-null    int64         
dtypes: datetime64[ns](2), int64(2)
memory usage: 10.0 KB


,ID,Inicio del rango,Fin del rango,Duración
0,169484,2023-12-01 14:00:00,2023-12-01 16:00:00,3
1,169484,2023-12-25 14:00:00,2023-12-25 17:00:00,4
2,169484,2024-01-05 15:00:00,2024-01-05 17:00:00,3
3,169484,2024-01-06 14:00:00,2024-01-06 17:00:00,4
4,169484,2024-01-09 14:00:00,2024-01-09 18:00:00,5


In [160]:
duración_ola_calor['Inicio del rango'] = pd.to_datetime(duración_ola_calor['Inicio del rango'])
duración_ola_calor['Fin del rango'] = pd.to_datetime(duración_ola_calor['Fin del rango'])

# Extraer mes y año
duración_ola_calor['Month'] = duración_ola_calor['Inicio del rango'].dt.month
duración_ola_calor['Year'] = duración_ola_calor['Inicio del rango'].dt.year

# Mostrar DataFrame con mes y año
print(duración_ola_calor[['ID', 'Inicio del rango', 'Fin del rango', 'Duración', 'Month', 'Year']])


         ID    Inicio del rango       Fin del rango  Duración  Month  Year
0    169484 2023-12-01 14:00:00 2023-12-01 16:00:00         3     12  2023
1    169484 2023-12-25 14:00:00 2023-12-25 17:00:00         4     12  2023
2    169484 2024-01-05 15:00:00 2024-01-05 17:00:00         3      1  2024
3    169484 2024-01-06 14:00:00 2024-01-06 17:00:00         4      1  2024
4    169484 2024-01-09 14:00:00 2024-01-09 18:00:00         5      1  2024
..      ...                 ...                 ...       ...    ...   ...
310  169529 2024-04-26 12:00:00 2024-04-26 17:00:00         6      4  2024
311  169529 2024-04-27 12:00:00 2024-04-27 18:00:00         7      4  2024
312  169529 2024-04-28 11:00:00 2024-04-28 18:00:00         8      4  2024
313  169529 2024-04-29 12:00:00 2024-04-29 18:00:00         7      4  2024
314  169529 2024-04-30 12:00:00 2024-04-30 15:00:00         4      4  2024

[315 rows x 6 columns]


In [161]:
# Obtener el año más reciente en el DataFrame
año_presente = duración_ola_calor['Year'].max()
año_anterior = año_presente - 1

# Filtrar por los meses de septiembre del año anterior a abril del año presente
filtro_meses = ((duración_ola_calor['Year'] == año_anterior) & (duración_ola_calor['Month'] >= 9)) | ((duración_ola_calor['Year'] == año_presente) & (duración_ola_calor['Month'] <= 4))
duración_ola_calor2 = duración_ola_calor
df_duracion_ola_calor_cultivo = duración_ola_calor2[filtro_meses]

In [162]:
df_duracion_ola_calor_cultivo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315 entries, 0 to 314
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ID                315 non-null    int64         
 1   Inicio del rango  315 non-null    datetime64[ns]
 2   Fin del rango     315 non-null    datetime64[ns]
 3   Duración          315 non-null    int64         
 4   Month             315 non-null    int32         
 5   Year              315 non-null    int32         
dtypes: datetime64[ns](2), int32(2), int64(2)
memory usage: 12.4 KB


In [163]:
# Filtrar valores de Duración mayores o iguales a 3
df_duracion_ola_calor_cultivo = df_duracion_ola_calor_cultivo[df_duracion_ola_calor_cultivo['Duración'] >= 3]

In [164]:
df_duracion_ola_calor_cultivo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315 entries, 0 to 314
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ID                315 non-null    int64         
 1   Inicio del rango  315 non-null    datetime64[ns]
 2   Fin del rango     315 non-null    datetime64[ns]
 3   Duración          315 non-null    int64         
 4   Month             315 non-null    int32         
 5   Year              315 non-null    int32         
dtypes: datetime64[ns](2), int32(2), int64(2)
memory usage: 12.4 KB


In [165]:
# Calcular el promedio de duración por estación
promedio_por_estacion_duracion_ola = df_duracion_ola_calor_cultivo.groupby('ID')['Duración'].mean()

# Mostrar el resultado
print("Promedio de duración por estación:")
print(promedio_por_estacion_duracion_ola)

# Convertir el diccionario en DataFrame
promedio_por_estacion_duracion_ola_calor = pd.DataFrame(list(promedio_por_estacion_duracion_ola.items()), columns=['ID', 'Promedio'])

# Establecer 'ID' como índice
promedio_por_estacion_duracion_ola_calor.set_index('ID', inplace=True)

# Mostrar el DataFrame resultante
print(promedio_por_estacion_duracion_ola_calor)

Promedio de duración por estación:
ID
169484    5.892308
169523    6.721519
169524    6.683168
169529    5.842857
Name: Duración, dtype: float64
        Promedio
ID              
169484  5.892308
169523  6.721519
169524  6.683168
169529  5.842857


**Duración promedio de ola de calor por mes**

In [167]:
# Agrupar por mes, año y ID y calcular el promedio de duración por estación
promedio_por_estacion_duracion_ola_mes = duración_ola_calor.groupby([duración_ola_calor['Inicio del rango'].dt.year,
                                                                    duración_ola_calor['Inicio del rango'].dt.month,
                                                                    'ID'])['Duración'].mean()

# Mostrar el resultado
print("Promedio de duración por estación:")
print(promedio_por_estacion_duracion_ola_mes)

# Convertir el diccionario en DataFrame
promedio_por_estacion_duracion_ola_mes = pd.DataFrame(list(promedio_por_estacion_duracion_ola_mes.items()), columns=['ID', 'Promedio'])

# Establecer 'ID' como índice
promedio_por_estacion_duracion_ola_mes.set_index('ID', inplace=True)

# Mostrar el DataFrame resultante
print(promedio_por_estacion_duracion_ola_mes)

Promedio de duración por estación:
Inicio del rango  Inicio del rango  ID    
2023              11                169524    3.750000
                  12                169484    3.500000
                                    169523    4.333333
                                    169524    4.875000
2024              1                 169484    3.714286
                                    169523    4.800000
                                    169524    5.235294
                                    169529    3.800000
                  2                 169484    4.600000
                                    169523    5.583333
                                    169524    5.555556
                                    169529    5.000000
                  3                 169484    6.833333
                                    169523    7.413793
                                    169524    7.689655
                                    169529    6.520000
                  4                 169484